In [ ]:
import collections
from datetime import datetime as dt
from datetime import timedelta
from edgar import Company
import graphviz
import json
from lxml import html
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import pandas_datareader as pdr
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import re
import requests
import sec_scraper
import statsmodels.api as sm
from sklearn import metrics, preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import ElasticNet, LinearRegression, LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV, TimeSeriesSplit, train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.svm import SVC
import time
import warnings
from yellowbrick.regressor import ResidualsPlot
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 100
plotly.offline.init_notebook_mode()
warnings.filterwarnings('ignore')

In [ ]:
# Functions for getting the CIK and Company Name from a SPAC ticker

def get_ticker_to_cik(write=False):
    # local copy: data/ticker_to_cik.txt
    ticker_to_cik = pd.read_csv('https://www.sec.gov/include/ticker.txt',
                                sep='\t', header=None, names=['ticker','cik'])
    if write:
        ticker_to_cik.to_csv('~/data/ticker_to_cik.csv', index=False)
    ticker_to_cik['ticker'] = ticker_to_cik.ticker.str.upper()
    ticker_to_cik['cik'] = ticker_to_cik.cik.astype(str)
    return ticker_to_cik

def get_cik_to_name(write=False):
    # local copy: data/cik_to_name.json
    cik_to_name = requests.get('https://www.sec.gov/files/company_tickers.json')
    cik_to_name = json.dumps(cik_to_name.json())
    cik_to_name = pd.read_json(cik_to_name).transpose()
    cik_to_name['ticker'] = cik_to_name['ticker'].str.upper()
    cik_to_name['cik_str'] = cik_to_name['cik_str'].astype(str)
    cik_to_name.rename(columns={'cik_str':'cik'}, inplace=True)
    return cik_to_name

def get_current_spacs(file_path_current, write=False):
    # existing current spac list
    df_spacs_existing = pd.read_csv(file_path_current)
    df_spacs_existing.drop_duplicates(inplace=True)
    
    # "spac traq" spac list
#     df_traq = pd.read_csv('https://docs.google.com/spreadsheets/d/14BY8snyHMbUReQVRgZxv4U4l1ak79dWFIymhrLbQpSo/'
#                              'export?gid=0&format=csv', header=2)
#     df_traq.columns = [x.replace('\n','') for x in df_traq.columns]
#     spac_traq_symbols = df_traq['Issuer Symbol']
    
    # "spac track" spac list
    path_spactrack = 'https://sheet2site.com/api/v3/index.php?key=1F7gLiGZP_F4tZgQXgEhsHMqlgqdSds3vO0-4hoL6ROQ&g=1&e=1&g=1'
    page = requests.get(path_spactrack)
    tree = html.fromstring(page.content)
    html_table = tree.xpath('//table[@class="table table-sm"]')
    str_table = html.etree.tostring(html_table[0])
    df_track = pd.read_html(str_table)[0]
    df_track = df_track[df_track['Status-Filter']!='Pre IPO']
    spac_track_symbols = df_track['SPAC Ticker-Filter']
    
    # combine
    combined_spacs = df_spacs_existing['Ticker'].append(spac_track_symbols)
    df_spacs_new = pd.DataFrame(combined_spacs, columns=['Ticker'])
    df_spacs_new.drop_duplicates(inplace=True)
    df_spacs_new.reset_index(inplace=True, drop=True)
    new_added_tickers = [x for x in df_spacs_new['Ticker'].tolist()
                         if x not in df_spacs_existing['Ticker'].tolist()]
    if len(new_added_tickers) > 0:
        print('newly added tickers:', new_added_tickers)
    if write:
        df_spacs_new.to_csv(file_path_current, index=False)
    return

def process_current_spacs(file_path_current, write=False):
    # current spac list
    spac_list_current = pd.read_csv(file_path_current)
    spac_list_current = spac_list_current.Ticker.unique()
    spac_list_current = pd.DataFrame(spac_list_current, columns=['Ticker'])
    
    # write to file
    if write==True:
        spac_list_current.to_csv('spac_list_current.csv', index=False)
        
    # get ticker to cik and cik to company name file, then merge
    ticker_to_cik = get_ticker_to_cik(write=write)
    cik_to_name = get_cik_to_name(write=write)
    spac_list_current = spac_list_current.merge(ticker_to_cik, how='left', left_on='Ticker', right_on='ticker')
    spac_list_current = spac_list_current.merge(cik_to_name[['cik','ticker','title']], how='left', on=['cik','ticker'])
    
    # some current spacs have not split from units to stock + warrants so ticker in sec different
    ticker_unit = pd.DataFrame(spac_list_current[spac_list_current.ticker.isna()]['Ticker'])
    ticker_unit['Ticker'] = ticker_unit.Ticker + 'U'
    ticker_unit = ticker_unit.merge(ticker_to_cik, how='left', left_on='Ticker', right_on='ticker')
    ticker_unit = ticker_unit.merge(cik_to_name[['cik','ticker','title']], how='left', on=['cik','ticker'])
    ticker_unit['Ticker'] = ticker_unit.Ticker.apply(lambda x: x[:-1] if not pd.isnull(x) else x)
    ticker_unit['ticker'] = ticker_unit.ticker.apply(lambda x: x[:-1] if not pd.isnull(x) else x)
    ticker_unit_other = pd.DataFrame(ticker_unit[ticker_unit.ticker.isna()]['Ticker'])
    ticker_unit_other['Ticker'] = ticker_unit_other.Ticker.apply(lambda x: x+'-UN')
    ticker_unit_other = ticker_unit_other.merge(ticker_to_cik, how='left', left_on='Ticker', right_on='ticker')
    ticker_unit_other = ticker_unit_other.merge(cik_to_name[['cik','ticker','title']], how='left', on=['cik','ticker'])
    ticker_unit_other['Ticker'] = ticker_unit_other.Ticker.apply(lambda x: x[:-3] if not pd.isnull(x) else x)
    ticker_unit_other['ticker'] = ticker_unit_other.ticker.apply(lambda x: x[:-3] if not pd.isnull(x) else x)
    ticker_unit = ticker_unit.append(ticker_unit_other)
    ticker_unit.dropna(inplace=True)
    
    # append tickers found using 'U' and '-UN'
    spac_list_current = spac_list_current[~spac_list_current.Ticker.isin(ticker_unit.Ticker)]
    spac_list_current = spac_list_current.append(ticker_unit)
    
    print('count current spacs:', len(spac_list_current))
    print('count nan in current spacs:', len(spac_list_current[spac_list_current.ticker.isna()]))
    
    return spac_list_current
    
def process_past_spacs(file_path_past, write=False):
    # past spac list (completed business combination)
    spac_list_past = pd.read_csv(file_path_past)
    spac_list_past.fillna('missing', inplace=True)
    spac_list_past['dupe_filter'] = spac_list_past['Old Ticker'] + spac_list_past['New Ticker']
    spac_list_past = spac_list_past[spac_list_past.dupe_filter.isin(spac_list_past.dupe_filter.unique())]
    spac_list_past.drop(columns=['dupe_filter'], inplace=True)
    
    # write to file
    if write==True:
        spac_list_past.to_csv('spac_list_past.csv', index=False)
        
    # get ticker to cik and cik to company name file, then merge
    ticker_to_cik = get_ticker_to_cik(write=write)
    cik_to_name = get_cik_to_name(write=write)
    spac_list_past = spac_list_past.merge(ticker_to_cik, how='left', left_on='New Ticker', right_on='ticker')
    spac_list_past = spac_list_past.merge(ticker_to_cik, how='left', left_on='Old Ticker', right_on='ticker')
    spac_list_past.rename(columns={'ticker_x':'ticker','cik_x':'cik','cik_y':'cik_old'}, inplace=True)
    spac_list_past.drop(columns='ticker_y', inplace=True)
    spac_list_past = spac_list_past.merge(cik_to_name[['cik','ticker','title']], how='left', on=['cik','ticker'])

    print('count past spacs:', len(spac_list_past))
    print('count nan in past spacs:', len(spac_list_past[spac_list_past.ticker.isna()]))
    
    return spac_list_past

In [ ]:
# Functions for pulling 8-K html, converting to text, grabbing date, and simple text match classifier

def basic_text_cleaning(text):
    text = text.replace('\n',' ').replace('\xa0',' ') # replace some unicode characters
    text = re.sub(' +', ' ', text) # remove extra spaces
    text = text.lower()
    return text

def get_forms_text(company_name, cik_id, form_type):
    c = sec_scraper.Company(company_name, cik_id, timeout=30)
    filings = c.get_all_filings(filing_type=form_type, no_of_documents=100)
    dates = [f.accepted_date.strftime('%Y-%m-%d %H:%M:%S') for f in filings]
    documents = [basic_text_cleaning(f.documents[0]) for f in filings]
    urls = [f.url for f in filings]
    print(company_name, cik_id)
    df = pd.DataFrame(list(zip(dates, documents, urls)), columns=['date','text','url'])
    df['form'] = '8-K'
    df['accepted_time'] = df.date
    df['date'] = df.date.apply(lambda x: x[0:10])
    df['url'] = df.url
    return df

def basic_text_match(df_form, substring):
    df_form[substring.replace(' ','_')+'_found'] = df_form.text.apply(lambda x: 1 if substring in x else 0)
    return df_form

In [ ]:
# Functions for grabbing and saving price data

def bulk_save_prices_to_csv(symbols, start_date, end_date, price_source, is_warrant=False):
    for ind, symbol in enumerate(symbols):
        print(symbol)
        df_prices = get_historical_prices(symbol=symbol,
                                          start_date=start_date,
                                          end_date=end_date,
                                          price_source=price_source,
                                          is_warrant=is_warrant)
        if df_prices is not None:
            df_prices = process_historical_prices(df_prices)
            df_prices = df_prices[[col for col in df_prices.columns if '%chg' not in col]]
            df_prices.to_csv('~/data/prices_'+price_source+'/daily_data/'+symbol+'_prices.csv', index=False)
        # TD and alphavantage limits query rate
        if ind%10==0 and ind!=0:
            time.sleep(30)
        
def run_bulk_save_daily_prices(spac_list_current, spac_list_past, price_source, is_warrant=False):
    # spacs missing price data entirely
    missing_past_spacs = ['missing', 'LCAH', 'FMCI1', 'CFCO']
    missing_current_spacs = ['ACNDU', 'BRLI', 'DFHT', 'DMYD', 'GOAC', 'IWAC',
                             'LCAH', 'MCAC', 'PANA', 'PSAC', 'PSTH', 'SSMC']
    
    symbols_past_new_ticker = [x for x in spac_list_past.ticker.unique().tolist() if str(x)!='nan']
    symbols_past_old_ticker = spac_list_past['Old Ticker'].unique().tolist()
    symbols_past_old_ticker = [x for x in symbols_past_old_ticker if x not in missing_past_spacs]
    symbols_current = [x for x in spac_list_current['Ticker'] if x not in missing_current_spacs]
    if is_warrant:
        symbols_past_new_ticker = [x + 'W' for x in symbols_past_new_ticker]
        symbols_past_old_ticker = [x + 'W' for x in symbols_past_old_ticker]
        symbols_current = [x + 'W' for x in symbols_current]
    bulk_save_prices_to_csv(symbols=symbols_past_new_ticker, start_date='2018-01-01',
                            end_date=dt.today().strftime('%Y-%m-%d'), price_source=price_source, is_warrant=is_warrant)
    bulk_save_prices_to_csv(symbols=symbols_past_old_ticker, start_date='2018-01-01',
                            end_date=dt.today().strftime('%Y-%m-%d'), price_source=price_source, is_warrant=is_warrant)
    bulk_save_prices_to_csv(symbols=symbols_current, start_date='2018-01-01',
                            end_date=dt.today().strftime('%Y-%m-%d'), price_source=price_source, is_warrant=is_warrant)

def get_td_key_locally():
    f = open("~/data/td_consumer_key.txt", "r")
    return f.read()

def millis_since_epoch_to_dt(x):
    return dt.fromtimestamp(float(x)/1000).strftime('%Y-%m-%d %H:%M:%S.%f')

def dt_to_millis_since_epoch(x):
    epoch = dt.utcfromtimestamp(0)
    return str(int((x-epoch).total_seconds()*1000))

def get_prices_td(symbol, start_date, end_date, minute_freq=False):
    TD_CONSUMER_KEY = get_td_key_locally()
    start_date_epoch = dt_to_millis_since_epoch(dt.strptime(start_date, '%Y-%m-%d'))
    end_date_epoch = dt_to_millis_since_epoch(dt.strptime(end_date, '%Y-%m-%d'))
    partial_url = 'https://api.tdameritrade.com/v1/marketdata/{symbol}/pricehistory?'
    # minute-to-minute prices (<2 months available)
    if minute_freq:
        endpoint = partial_url+'periodType=day&frequencyType=minute&startDate={start_date}&endDate={end_date}'
    # daily open-close prices
    else:
        endpoint = partial_url+'periodType=month&frequencyType=daily&startDate={start_date}&endDate={end_date}'
    full_url = endpoint.format(symbol=symbol,
                               start_date=start_date_epoch,
                               end_date=end_date_epoch)
    page = requests.get(url=full_url, params={'apikey' : TD_CONSUMER_KEY})
    content = json.loads(page.content)
    try:
        df_td_prices = pd.DataFrame.from_dict(content['candles'])
    except:
        return None
    if len(df_td_prices)==0:
        return None
    df_td_prices['datetime'] = df_td_prices.datetime.apply(lambda x: millis_since_epoch_to_dt(x))
    df_td_prices['date'] = df_td_prices.datetime.apply(lambda x: x[0:10])
    df_td_prices['ticker'] = symbol
    return df_td_prices

def get_historical_prices(symbol, start_date, end_date, price_source, is_warrant=False):
    print('getting prices from', start_date, 'to', end_date, 'for', symbol)
    if price_source=='td':
        # get prices using td api
        df_prices = get_prices_td(symbol, start_date, end_date)
        
        # td symbol format for some warrants is ___.WS
        if df_prices is None and is_warrant:
            df_prices = get_prices_td(symbol[:-1]+'.WS', start_date, end_date)
    else:
        start_split = start_date.split('-')
        end_split = end_date.split('-')
        start = dt(int(start_split[0]), int(start_split[1]), int(start_split[2]))
        end = dt(int(end_split[0]), int(end_split[1]), int(end_split[2]))
        try:
            # be careful with missing/limited data in yahoo data
            if price_source=='yahoo':
                df_prices = pdr.data.DataReader(name=symbol, data_source='yahoo', start=start, end=end)
            # alphavantage seems to be most reliable. 5 calls per minute and 500 calls per day
            if price_source=='alphavantage':
                df_prices = pdr.data.DataReader(name=symbol, data_source='av-daily', start=start, end=end,
                                                api_key='BDB9WJQRCZKINCLD')
            # iex has extremely limited api calls
            if price_source=='iex':
                df_prices = pdr.data.DataReader(name=symbol, data_source='iex', start=start, end=end,
                                                api_key='pk_970dfff359894b15a056cf677c02e11f')
            df_prices.reset_index(inplace=True)
            df_prices.rename(columns={'index':'date'}, inplace=True)
        except:
            return None
    return df_prices

def process_historical_prices(df_prices, conservative_est=False):
    df_prices.columns = df_prices.columns.str.lower()
    df_prices['date'] = df_prices.date.astype(str)
    
    # find price on t+# date (can't use shift operator b/c some (business) days missing
    # e.g. ACAMW 2019-04-15->2019-04-16->2019-06-05)
    def get_price_date_t(date, t, open_or_close):
        date = dt.strptime(date, '%Y-%m-%d')
        date_range = [date + timedelta(days=i) for i in range(0,30)]
        date_range_no_weekend = [date.strftime('%Y-%m-%d') for date in date_range if date.weekday() not in [5,6]]
        date_t = date_range_no_weekend[t]
        # assume missing day means no volume that day
        if date_t > df_prices.iloc[-1]['date']:
            return np.nan
        return df_prices[df_prices['date']<=date_t].iloc[-1][open_or_close]
    
    # add price columns
    df_prices['open_t+1'] = df_prices.date.apply(lambda x: get_price_date_t(x, 1, 'open'))
    for i in range(1,13):
        df_prices['close_t+'+str(i)] = df_prices.date.apply(lambda x: get_price_date_t(x, i, 'close'))
        # for conservative estimate, mark return from open_t+1 to close_t+#
        if conservative_est:
            df_prices['open_close_t+'+str(i)+'_%chg'] = (df_prices['close_t+'+str(i)] - df_prices['open_t+1']) / df_prices['open_t+1']
        # if not conservative estimate, mark return from open_t to close_t+#
        else:
            df_prices['open_close_t+'+str(i)+'_%chg'] = (df_prices['close_t+'+str(i)] - df_prices['open']) / df_prices['open']
    df_prices = df_prices.round(2)
    print('output min date:', df_prices.date.min())
    print('output max date:', df_prices.date.max())
    return df_prices

# df_prices = pd.read_csv('data/prices_td/daily_data/CCXXW_prices.csv')
# df_prices = df_prices[['open','high','low','close','volume','datetime','date','ticker']]
# df_prices = process_historical_prices(df_prices, conservative_est=True)
# df_prices

In [ ]:
# Functions outputting dataframe of 8-Ks, classification, and 1,3,5,7 day returns

def agg_spac_returns(spac_list, price_source, is_warrant=False, write=False, conservative_est=False):
    df_returns_agg = pd.DataFrame()
    count_missing_8K = 0
    for ind in range(0, len(spac_list)):
        row = spac_list.iloc[ind]
        print('index:', ind)
        print(row.ticker)
        
        broken_current_spacs = ['GNRS','KBLM','LGC','LIVE','NOVS']
        if is_warrant:
            broken_current_spacs = [x + 'W' for x in broken_current_spacs]
        if row.ticker in broken_current_spacs:
            print('in broken spac list, skipping...')
            continue

        # get form 8Ks
        df_form_8K = get_forms_text(company_name=row.title, cik_id=row.cik, form_type='8-K')
        if df_form_8K is None or len(df_form_8K)==0:
            print('no 8Ks found, trying cik_old...')
            if 'cik_old' in row.index:
                df_form_8K = get_forms_text(company_name=row.title, cik_id=row.cik_old, form_type='8-K')
        if df_form_8K is None or len(df_form_8K)==0:
            print('no 8Ks found, skipping...\n')
            count_missing_8K = count_missing_8K + 1
            continue
        else:
            if write:
                df_form_8K.to_csv('~/data/sec_filings_df/'+row.ticker+'_sec_forms.csv', index=False)
            # simple classifier
            df_form_8K = basic_text_match(df_form_8K, 'letter of intent')
            df_form_8K = basic_text_match(df_form_8K, 'business combination agreement')

        # get stock or warrant prices
        try:
            # load saved data
            df_prices = pd.read_csv('~/data/prices_'+price_source+'/daily_data/'+row.ticker+'_prices.csv') 
            print('price data min date:', df_prices.date.min())
            print('price data max date:', df_prices.date.max())
        except:
            df_prices = get_historical_prices(symbol=row.ticker,
                                              start_date='2018-01-01',
                                              end_date=dt.today().strftime('%Y-%m-%d'),
                                              price_source=price_source,
                                              is_warrant=is_warrant)
        if df_prices is None:
            print('prices for', row.ticker, 'not found. skipping...\n')
            continue
        df_prices = process_historical_prices(df_prices, conservative_est)
        
        # add completion (de-spac) date price if past spac
        if 'Completion Date' in spac_list.columns:
            if is_warrant:
                ticker_match = row.ticker[:-1]
            else:
                ticker_match = row.ticker
            if len(spac_list[spac_list['New Ticker']==ticker_match])!=0:
                completion_date = spac_list[spac_list['New Ticker']==ticker_match]['Completion Date'].iloc[0]
            elif len(spac_list[spac_list['Old Ticker']==ticker_match])!=0:
                completion_date = spac_list[spac_list['Old Ticker']==ticker_match]['Completion Date'].iloc[0]
            else:
                completion_date = np.nan
                print('could not find completion (de-spac) date')
            print('completion (de-spac) date:', completion_date)
            completion_row = df_prices[df_prices['date']==completion_date]
            if len(completion_row)==0 or pd.isna(completion_date):
                df_prices['open_completion_%chg'] = np.nan
                print('could not find price for completion (de-spac) date')
            else:
                completion_price = completion_row['close'].iloc[0]
                df_prices['completion_price'] = completion_price
                print('completion (de-spac) price:', completion_price)
                # for conservative estimate, mark return from open_t+1
                if conservative_est:
                    df_prices['open_completion_%chg'] = (df_prices['completion_price'] - df_prices['open_t+1']) / df_prices['open_t+1']
                # if not conservative estimate, mark return from open_t
                else:
                    df_prices['open_completion_%chg'] = (df_prices['completion_price'] - df_prices['open']) / df_prices['open']  
                # set to nan if date >= completion_date
                df_prices['open_completion_%chg'] = np.where(df_prices['date']>=completion_date, np.nan, np.round(df_prices['open_completion_%chg'], 2))
        else:
            df_prices['open_completion_%chg'] = np.nan

        # merge sec forms and price data on date
        cols = ['date']
        cols_pct_chg = [col for col in df_prices.columns if '%chg' in col]
        cols.extend(cols_pct_chg)
        df_returns = df_form_8K.merge(df_prices[cols], how='left', on='date') # some dates missing
        display(df_returns)

        # append
        if len(df_returns)!=0:
            df_returns['symbol'] = row.ticker
        df_returns_agg = df_returns_agg.append(df_returns)
        
        # sec site sometimes will timeout
        if ind%10==0 and ind!=0:
            time.sleep(30) # set to at least 120 loading sec forms from scratch
        
    # drop duplicates on date + symbol
    # todo: if date has multipled 8-Ks, concatenate instead of dropping
    print('count sec forms before dropping duplicates:', len(df_returns_agg))
    df_returns_agg = df_returns_agg.drop_duplicates(subset=['date','symbol'])
    print('count sec forms after dropping duplicates:', len(df_returns_agg))
    
    print('\n##############\ncount symbols missing 8-Ks:', count_missing_8K, '\n##############\n')
    
    return df_returns_agg

def load_all_spacs(use_saved_df=True, write=False, warrants_only=False, conservative_est=False):
    if conservative_est:
        path_returns = '~/data/returns/conservative/'
    else:
        path_returns = '~/data/returns/'
    if use_saved_df:
        df_returns_past_warrants_newticker = pd.read_csv(path_returns+'df_returns_past_warrants_newticker.csv')
        df_returns_past_warrants_oldticker = pd.read_csv(path_returns+'df_returns_past_warrants_oldticker.csv')
        df_returns_current_warrants = pd.read_csv(path_returns+'df_returns_current_warrants.csv')
        if warrants_only:
            return df_returns_past_warrants_newticker, df_returns_past_warrants_oldticker, df_returns_current_warrants
        df_returns_past = pd.read_csv(path_returns+'df_returns_past.csv')
        df_returns_current = pd.read_csv(path_returns+'df_returns_current.csv')
        return df_returns_past, df_returns_current, df_returns_past_warrants_newticker, df_returns_past_warrants_oldticker, df_returns_current_warrants
    else:
        # get returns following 8-Ks for past spac warrants (new and old tickers)
        spac_list_past_warrants_newticker = spac_list_past.copy()
        spac_list_past_warrants_newticker['ticker'] = spac_list_past_warrants_newticker['New Ticker'] + 'W' # warrants
        df_returns_past_warrants_newticker = agg_spac_returns(spac_list_past_warrants_newticker,
                                                              price_source='td',
                                                              is_warrant=True,
                                                              write=write,
                                                              conservative_est=conservative_est)
        spac_list_past_warrants_oldticker = spac_list_past.copy()
        spac_list_past_warrants_oldticker['ticker'] = spac_list_past_warrants_oldticker['Old Ticker'] + 'W' # warrants
        df_returns_past_warrants_oldticker = agg_spac_returns(spac_list_past_warrants_oldticker,
                                                              price_source='td',
                                                              is_warrant=True,
                                                              write=write,
                                                              conservative_est=conservative_est)

        # get returns following 8-Ks for current spac warrants
        spac_list_current_warrants = spac_list_current.copy()
        spac_list_current_warrants['ticker'] = spac_list_current_warrants.Ticker + 'W' # warrants
        df_returns_current_warrants = agg_spac_returns(spac_list_current_warrants,
                                                       price_source='td',
                                                       is_warrant=True,
                                                       write=write,
                                                       conservative_est=conservative_est)
        if warrants_only:
            return df_returns_past_warrants_newticker, df_returns_past_warrants_oldticker, df_returns_current_warrants
        
        # get returns following 8-Ks for past spacs
        df_returns_past = agg_spac_returns(spac_list_past,
                                           price_source='td',
                                           is_warrant=False,
                                           write=write,
                                           conservative_est=conservative_est)

        # get returns following 8-Ks for current spacs
        df_returns_current = agg_spac_returns(spac_list_current,
                                              price_source='td',
                                              is_warrant=False,
                                              write=write,
                                              conservative_est=conservative_est)

        if write:
            df_returns_past.to_csv(path_returns+'df_returns_past.csv', index=False)
            df_returns_current.to_csv(path_returns+'df_returns_current.csv', index=False)
            df_returns_past_warrants_newticker.to_csv(path_returns+'df_returns_past_warrants_newticker.csv', index=False)
            df_returns_past_warrants_oldticker.to_csv(path_returns+'df_returns_past_warrants_oldticker.csv', index=False)
            df_returns_current_warrants.to_csv(path_returns+'df_returns_current_warrants.csv', index=False)
            
    return df_returns_past, df_returns_current, df_returns_past_warrants_newticker, df_returns_past_warrants_oldticker, df_returns_current_warrants

In [ ]:
def get_spac_track_table(path_spactrack):
    page = requests.get(path_spactrack)
    tree = html.fromstring(page.content)
    html_table = tree.xpath('//table[@class="table table-sm"]')
    str_table = html.etree.tostring(html_table[0])
    df_track = pd.read_html(str_table)[0]
    return df_track

def process_past_spacs(file_path_past, write=False):
    # past spac list (completed business combination)
    spac_list_past = pd.read_csv(file_path_past)
    spac_list_past.fillna('missing', inplace=True)
    spac_list_past['dupe_filter'] = spac_list_past['Old Ticker'] + spac_list_past['New Ticker']
    spac_list_past = spac_list_past[spac_list_past.dupe_filter.isin(spac_list_past.dupe_filter.unique())]
    spac_list_past.drop(columns=['dupe_filter'], inplace=True)
    
    # write to file
    if write==True:
        spac_list_past.to_csv('spac_list_past.csv', index=False)
        
    # get ticker to cik and cik to company name file, then merge
    ticker_to_cik = get_ticker_to_cik(write=write)
    cik_to_name = get_cik_to_name(write=write)
    spac_list_past = spac_list_past.merge(ticker_to_cik, how='left', left_on='New Ticker', right_on='ticker')
    spac_list_past = spac_list_past.merge(ticker_to_cik, how='left', left_on='Old Ticker', right_on='ticker')
    spac_list_past.rename(columns={'ticker_x':'ticker','cik_x':'cik','cik_y':'cik_old'}, inplace=True)
    spac_list_past.drop(columns='ticker_y', inplace=True)
    spac_list_past = spac_list_past.merge(cik_to_name[['cik','ticker','title']], how='left', on=['cik','ticker'])

    print('count past spacs:', len(spac_list_past))
    print('count nan in past spacs:', len(spac_list_past[spac_list_past.ticker.isna()]))
    
    # merge "spac track" completion date
    df_track_past = get_spac_track_table(path_spactrack='https://sheet2site.com/api/v3/index.php?key=1OMSgBXCcW4W-5h20eZYb1RCmkuO6Sf299foDPUF7Fc0&g=1&e=1')
    spac_list_past = spac_list_past.merge(df_track_past[['Post-SPAC Ticker Symbol','Completion Date']], how='left', left_on='New Ticker', right_on='Post-SPAC Ticker Symbol')
    spac_list_past = spac_list_past[['Old Ticker','New Ticker','Completion Date','Completion Year','ticker','cik','cik_old','title']]
    
    return spac_list_past

In [ ]:
# update current spac list
# get_current_spacs(file_path_current='data/spac_list_current.csv', write=True)

# load current and past spac lists
spac_list_current = process_current_spacs(file_path_current='~/data/spac_list_current.csv', write=False)
spac_list_past = process_past_spacs(file_path_past='~/data/spac_list_past.csv', write=False)

In [ ]:
# bulk save daily price data (API limits)...run this occasionally to update daily prices
# run_bulk_save_daily_prices(spac_list_current, spac_list_past, price_source='td', is_warrant=False)
# run_bulk_save_daily_prices(spac_list_current, spac_list_past, price_source='td', is_warrant=True)

In [ ]:
# load returns for all spacs (stock and warrants) with returns calculated from open on form accepted date
# df_returns_past, df_returns_current, df_returns_past_warrants_newticker, df_returns_past_warrants_oldticker, df_returns_current_warrants = load_all_spacs(use_saved_df=True, write=False, warrants_only=False, conservative_est=False)

# load returns for all spacs (warrants only) with returns calculated from open on form accepted date
# df_returns_past_warrants_newticker, df_returns_past_warrants_oldticker, df_returns_current_warrants = load_all_spacs(use_saved_df=True, write=False, warrants_only=True, conservative_est=False)

# load returns for all spacs (stock and warrants) with returns calculated from open on form accepted date + 1 day (conservative)
df_returns_past, df_returns_current, df_returns_past_warrants_newticker, df_returns_past_warrants_oldticker, df_returns_current_warrants = load_all_spacs(use_saved_df=True, write=False, warrants_only=False, conservative_est=True)

# load returns for all spacs (warrants only) with returns calculated from open on form accepted date + 1 day (conservative)
# df_returns_past_warrants_newticker, df_returns_past_warrants_oldticker, df_returns_current_warrants = load_all_spacs(use_saved_df=True, write=False, warrants_only=True, conservative_est=True)


print('count current spacs with 8-Ks found:', len(df_returns_current.symbol.unique()))
print('count past spacs with 8-Ks found', len(df_returns_past.symbol.unique()))

## Analysis

In [ ]:
# Functions for backtest
# note .123 is 12.3% in output

def compute_mean_returns(df_returns, corrupt_symbols, includes_hourly=False):
    if len(corrupt_symbols)!=0:
        print('Symbols removed:', corrupt_symbols)
    if includes_hourly:
        agg_dict = {'accepted_time_t+1hr_%chg':'mean','accepted_time_t+3hr_%chg':'mean',
                    'accepted_time_t+5hr_%chg':'mean','accepted_time_t+7hr_%chg':'mean',
                    'accepted_time_t+10hr_%chg':'mean','accepted_time_t+12hr_%chg':'mean',
                    'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
                    'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean',
                    'open_close_t+10_%chg':'mean','open_close_t+12_%chg':'mean',
                    'open_completion_%chg':'mean'}
    else:
        agg_dict = {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
                    'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean',
                    'open_close_t+10_%chg':'mean','open_close_t+12_%chg':'mean',
                    'open_completion_%chg':'mean'}
    df_mean_returns = df_returns[~df_returns.symbol.isin(corrupt_symbols)].groupby('symbol').agg(agg_dict)
    df_mean_returns.dropna(inplace=True)
    print('Count:', len(df_mean_returns))
    return np.round(df_mean_returns, 3)

def summary_statistics(df_returns, corrupt_symbols):
    if len(corrupt_symbols)!=0:
        print('Symbols removed:', corrupt_symbols)
    return np.round(df_returns[~df_returns.symbol.isin(corrupt_symbols)].describe(), 3)

def compute_mean_returns_LOI(df_returns, corrupt_symbols, includes_hourly=False):
    if len(corrupt_symbols)!=0:
        print('Symbols removed:', corrupt_symbols)
    if includes_hourly:
        agg_dict = {'accepted_time_t+1hr_%chg':'mean','accepted_time_t+3hr_%chg':'mean',
                    'accepted_time_t+5hr_%chg':'mean','accepted_time_t+7hr_%chg':'mean',
                    'accepted_time_t+10hr_%chg':'mean','accepted_time_t+12hr_%chg':'mean',
                    'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
                    'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean',
                    'open_close_t+10_%chg':'mean','open_close_t+12_%chg':'mean'}
    else:
        agg_dict = {'open_close_t+1_%chg':'mean','open_close_t+3_%chg':'mean',
                    'open_close_t+5_%chg':'mean','open_close_t+7_%chg':'mean',
                    'open_close_t+10_%chg':'mean','open_close_t+12_%chg':'mean'}
    return np.round(df_returns[(~df_returns.symbol.isin(corrupt_symbols)) &
                               (df_returns.letter_of_intent_found==1)].groupby('symbol').agg(agg_dict), 3)

Past SPACs (Stock)

In [ ]:
print('mean returns for past spac stocks')
compute_mean_returns(df_returns_past, corrupt_symbols=['ACEL'])

In [ ]:
print('summary stats for returns in past spac stocks')
summary_statistics(df_returns_past, corrupt_symbols=['ACEL'])

In [ ]:
print('mean returns for past spac stocks (LOI text match)')
compute_mean_returns_LOI(df_returns_past, corrupt_symbols=['ACEL'])

Current SPACs (Stock)

In [ ]:
print('mean returns for current spac stocks')
compute_mean_returns(df_returns_current, corrupt_symbols=[])

In [ ]:
print('summary stats for returns in current spac stocks')
summary_statistics(df_returns_current, corrupt_symbols=[])

In [ ]:
print('mean returns for current spac stocks (LOI text match)')
compute_mean_returns_LOI(df_returns_current, corrupt_symbols=[])

Past SPAC Warrants

In [ ]:
print('mean returns for past spac warrants')
compute_mean_returns(df_returns_past_warrants_newticker, corrupt_symbols=['LAZYW'])

In [ ]:
print('summary stats for returns in past spac warrants')
summary_statistics(df_returns_past_warrants_newticker, corrupt_symbols=['LAZYW'])

In [ ]:
print('mean returns for past spac warrants (LOI text match)')
compute_mean_returns_LOI(df_returns_past_warrants_newticker, corrupt_symbols=['LAZYW'])

Current SPAC Warrants

In [ ]:
print('mean returns for current spac warrants')
compute_mean_returns(df_returns_current_warrants, corrupt_symbols=[])

In [ ]:
print('summary stats for returns in current spac warrants')
summary_statistics(df_returns_current_warrants, corrupt_symbols=[])

In [ ]:
print('mean returns for current spac warrants (LOI text match)')
compute_mean_returns_LOI(df_returns_current_warrants, corrupt_symbols=[])

In [ ]:
df_returns_all_warrants = df_returns_past_warrants_newticker.append(df_returns_current_warrants)
df_returns_all_warrants = df_returns_all_warrants.replace([np.inf, -np.inf], np.nan)
print('mean returns for all spac warrants')
compute_mean_returns(df_returns_all_warrants, corrupt_symbols=['LAZYW'])

In [ ]:
print('summary stats for returns all spac warrants')
summary_statistics(df_returns_all_warrants, corrupt_symbols=['LAZYW'])

Results using minute-by-minute data

In [ ]:
def create_dict_minute_prices(df_returns, min_minute_date='2020-06-01', write=False):
    price_dict = dict()
    for symbol in df_returns.symbol.unique():
        minute_prices = get_prices_td(symbol=symbol, start_date=min_minute_date,
                                      end_date=dt.today().strftime('%Y-%m-%d'), minute_freq=True)
        if minute_prices is None: # try .WS instead of W
            minute_prices = get_prices_td(symbol=symbol[:-1]+'.WS', start_date=min_minute_date,
                                          end_date=dt.today().strftime('%Y-%m-%d'), minute_freq=True)
        if minute_prices is not None:
            # add to dict
            minute_prices['datetime'] = minute_prices.datetime.apply(lambda x: x[0:19])
            price_dict[symbol] = minute_prices
            
            # save to file
            if write:
                path_minute = '~/data/prices_td/minute_data/'+symbol+'.csv'
                if os.path.exists(path_minute):
                    saved_minute_prices = pd.read_csv(path_minute)
                    minute_prices = saved_minute_prices.append(minute_prices)
                    minute_prices.drop_duplicates(subset=['datetime'], keep='first', inplace=True)
                minute_prices.to_csv(path_minute, index=False)
        else:
            print(symbol, 'missing')
    return price_dict

def get_price_at_time(x, n, price_dict, is_hour, return_price=False):
    if x.symbol not in price_dict:
        return None
    symbol_minute_prices = price_dict[x.symbol]
    if is_hour:
        time_dt = dt.strptime(x.accepted_time, '%Y-%m-%d %H:%M:%S') + timedelta(hours=n)
        symbol_minute_prices = symbol_minute_prices[symbol_minute_prices.datetime>time_dt.strftime('%Y-%m-%d %H:%M:%S')]
    else:
        time_dt = dt.strptime(x.accepted_time, '%Y-%m-%d %H:%M:%S') + timedelta(days=n)
        symbol_minute_prices = symbol_minute_prices[symbol_minute_prices.datetime>time_dt.strftime('%Y-%m-%d %H:%M:%S')]
    if len(symbol_minute_prices)!=0:
        price_at_time = symbol_minute_prices.iloc[0]['open']
        if return_price:
            return price_at_time
        pct_return = (price_at_time - x.accepted_time_price) / x.accepted_time_price
        return pct_return
    return None

def get_returns_with_minute_data(past, current, min_minute_date='2020-06-01', write=False):
    # subset all 8-Ks where there exists minute data
    df_returns_minute = past.append(current)
    df_returns_minute = df_returns_minute[df_returns_minute.date>=min_minute_date]
    df_returns_minute.reset_index(inplace=True, drop=True)

    # create dictionary of symbol, minute prices df
    price_dict = create_dict_minute_prices(df_returns=df_returns_minute, min_minute_date=min_minute_date, write=write)

    # compute returns
    df_returns_minute['accepted_time_price'] = df_returns_minute.apply(lambda x: get_price_at_time(x, price_dict=price_dict, n=0, is_hour=True, return_price=True), axis=1)
    for hr in range(1,13):
        df_returns_minute['accepted_time_t+'+str(hr)+'hr_%chg'] = df_returns_minute.apply(lambda x: get_price_at_time(x, price_dict=price_dict, n=hr, is_hour=True), axis=1)
    for day in range(1,13):
        df_returns_minute['accepted_time_t+'+str(day)+'_%chg'] = df_returns_minute.apply(lambda x: get_price_at_time(x, price_dict=price_dict, n=day, is_hour=False), axis=1)

    # stats
    print('mean returns for current spacs (using minute data)')
    display(compute_mean_returns(df_returns_minute, corrupt_symbols=[], includes_hourly=True).sort_values(by='accepted_time_t+1hr_%chg'))

    print('summary stats for returns in current spacs (using minute data)')
    cols = ['symbol','accepted_time_t+1hr_%chg','accepted_time_t+3hr_%chg','accepted_time_t+5hr_%chg','accepted_time_t+7hr_%chg','accepted_time_t+10hr_%chg','accepted_time_t+12hr_%chg',
            'accepted_time_t+1_%chg','accepted_time_t+3_%chg','accepted_time_t+5_%chg','accepted_time_t+7_%chg','accepted_time_t+10_%chg','accepted_time_t+12_%chg']
    display(summary_statistics(df_returns_minute[cols], corrupt_symbols=[]))
    print(summary_statistics(df_returns_minute[cols], corrupt_symbols=[]).loc['mean'],'\n')
    print(summary_statistics(df_returns_minute[cols], corrupt_symbols=[]).loc['std'],'\n')

    print('mean returns for current spacs (LOI text match) (using minute data)')
    display(compute_mean_returns_LOI(df_returns_minute, corrupt_symbols=[], includes_hourly=True).sort_values(by='accepted_time_t+1hr_%chg'))
    
    return df_returns_minute

In [ ]:
print('Stock')
df_returns_all_stocks_minute = get_returns_with_minute_data(past=df_returns_past,
                                                            current=df_returns_current,
                                                            min_minute_date='2020-06-01',
                                                            write=False)

In [ ]:
print('Warrants')
df_returns_all_warrants_minute = get_returns_with_minute_data(past=df_returns_past_warrants_newticker,
                                                              current=df_returns_current_warrants,
                                                              min_minute_date='2020-06-01',
                                                              write=False)

Study minute by minute LOI examples

In [ ]:
def plot_price(symbol, prices):
    layout = go.Layout(
        title = symbol+' LOI',
        xaxis = dict(title='Time'),
        yaxis = dict(title='Price'),
    )
    data = []
    trace = go.Scatter(
        x = prices.datetime,
        y = prices.open,
    )
    data.append(trace)
    fig = dict(data=data, layout=layout)
    iplot(fig)
    
def plot_return(symbol, prices):
    layout = go.Layout(
        title = symbol+' Return After LOI 8-K Released',
        xaxis = dict(title='Time'),
        yaxis = dict(title='Return')
    )
    data = []
    trace = go.Scatter(
        x = prices.datetime,
        y = prices['%return'],
    )
    data.append(trace)
    fig = dict(data=data, layout=layout)
    iplot(fig)
    
def plot_volume(symbol, prices):
    layout = go.Layout(
        title = symbol+' Volume After LOI 8-K Released',
        xaxis = dict(title='Time'),
        yaxis = dict(title='Volume')
    )
    data = []
    trace = go.Scatter(
        x = prices.datetime,
        y = prices['volume'],
    )
    data.append(trace)
    fig = dict(data=data, layout=layout)
    iplot(fig)

In [ ]:
display(df_returns_current_warrants[(df_returns_current_warrants.symbol=='OPESW') &
                                    (df_returns_current_warrants.letter_of_intent_found==1)])
accepted_time = '2020-06-09 16:01:19'
print('accepted time:', accepted_time)
minute_prices = pd.read_csv('~/data/prices_td/minute_data/OPESW.csv')
minute_prices = minute_prices[(minute_prices['date']>='2020-06-08') & (minute_prices['date']<='2020-06-13')]
minute_prices['datetime'] = minute_prices.datetime.apply(lambda x: x[0:19])
plot_price('OPESW', minute_prices)

minute_prices_post = minute_prices[minute_prices.datetime>accepted_time]
minute_prices_post.reset_index(drop=True, inplace=True)
price_before_8K = minute_prices_post.loc[0,'open']
minute_prices_post['%return'] = (minute_prices_post.open - price_before_8K) / price_before_8K
plot_return('OPESW', minute_prices_post)
plot_volume('OPESW', minute_prices_post)

In [ ]:
display(df_returns_current_warrants[(df_returns_current_warrants.symbol=='BMRGW') &
                                    (df_returns_current_warrants.letter_of_intent_found==1)])
accepted_time = '2020-06-24 13:24:44'
print('accepted time:', accepted_time)
minute_prices = pd.read_csv('~/data/prices_td/minute_data/BMRGW.csv')
minute_prices = minute_prices[(minute_prices['date']>='2020-06-24') & (minute_prices['date']<='2020-06-30')]
minute_prices['datetime'] = minute_prices.datetime.apply(lambda x: x[0:19])
plot_price('BMRGW', minute_prices)

minute_prices_post = minute_prices[minute_prices.datetime>accepted_time]
minute_prices_post.reset_index(drop=True, inplace=True)
price_before_8K = minute_prices_post.loc[0,'open']
minute_prices_post['%return'] = (minute_prices_post.open - price_before_8K) / price_before_8K
plot_return('BMRGW', minute_prices_post)
plot_volume('BMRGW', minute_prices_post)

In [ ]:
display(df_returns_current_warrants[(df_returns_current_warrants.symbol=='SAMAW') &
                                    (df_returns_current_warrants.letter_of_intent_found==1)])
accepted_time = '2020-06-01 09:37:22'
print('accepted time:', accepted_time)
minute_prices = pd.read_csv('~/data/prices_td/minute_data/SAMAW.csv')
minute_prices = minute_prices[(minute_prices['date']>='2020-06-01') & (minute_prices['date']<='2020-06-06')]
minute_prices['datetime'] = minute_prices.datetime.apply(lambda x: x[0:19])
plot_price('SAMAW', minute_prices)

minute_prices_post = minute_prices[minute_prices.datetime>accepted_time]
minute_prices_post.reset_index(drop=True, inplace=True)
price_before_8K = minute_prices_post.loc[0,'open']
minute_prices_post['%return'] = (minute_prices_post.open - price_before_8K) / price_before_8K
plot_return('SAMAW', minute_prices_post)
plot_volume('SAMAW', minute_prices_post)

In [ ]:
display(df_returns_current_warrants[(df_returns_current_warrants.symbol=='NFINW') &
                                    (df_returns_current_warrants.letter_of_intent_found==1)])
accepted_time = '2020-06-29 09:00:22'
print('accepted time:', accepted_time)
minute_prices = pd.read_csv('~/data/prices_td/minute_data/NFINW.csv')
minute_prices = minute_prices[(minute_prices['date']>='2020-06-29') & (minute_prices['date']<='2020-07-03')]
minute_prices['datetime'] = minute_prices.datetime.apply(lambda x: x[0:19])
plot_price('NFINW', minute_prices)

minute_prices_post = minute_prices[minute_prices.datetime>accepted_time]
minute_prices_post.reset_index(drop=True, inplace=True)
price_before_8K = minute_prices_post.loc[0,'open']
minute_prices_post['%return'] = (minute_prices_post.open - price_before_8K) / price_before_8K
plot_return('NFINW', minute_prices_post)
plot_volume('NFINW', minute_prices_post)

Performance Over Time

In [ ]:
df_returns_all = df_returns_past.append(df_returns_current)
df_returns_all.sort_values(by='date', inplace=True)
df_returns_all.set_index('date')['open_close_t+1_%chg'].cumsum().plot(label='Open-Close T+1')
df_returns_all.set_index('date')['open_close_t+3_%chg'].cumsum().plot(label='Open-Close T+3')
df_returns_all.set_index('date')['open_close_t+5_%chg'].cumsum().plot(label='Open-Close T+5')
df_returns_all.set_index('date')['open_close_t+7_%chg'].cumsum().plot(label='Open-Close T+7')
df_returns_all.set_index('date')['open_close_t+10_%chg'].cumsum().plot(label='Open-Close T+10')
df_returns_all.set_index('date')['open_close_t+12_%chg'].cumsum().plot(label='Open-Close T+12')
df_returns_all.set_index('date')['open_completion_%chg'].cumsum().plot(label='Open-Completion')
plt.title('Cumulative Return on All 8-Ks, All SPAC Stocks');
plt.ylabel('Cumulative Return in Percentage Points (10 = 1000%)');
plt.xticks(rotation=45);
plt.legend();

In [ ]:
df_returns_all = df_returns_past.append(df_returns_current)
df_returns_all.sort_values(by='date', inplace=True)
plot_col_names = ['open_close_t+1_%chg', 'open_close_t+3_%chg', 'open_close_t+5_%chg', 'open_close_t+7_%chg',
                  'open_close_t+10_%chg', 'open_close_t+12_%chg', 'open_completion_%chg']
df_cumsum = df_returns_all.set_index('date')[plot_col_names].cumsum()
min_date = df_cumsum[~df_cumsum['open_close_t+1_%chg'].isna()].index.min()
df_cumsum = df_cumsum[df_cumsum.index>=min_date]
fig = go.Figure()
fig.update_layout(title='Cumulative Return Trading on All 8-Ks: SPAC Stock',
                  xaxis_title='Date',
                  yaxis_title='Cumulative Return in Percentage Points (10 = 1000%)')
for plot_col_name in plot_col_names:
    fig.add_trace(go.Scatter(x=df_cumsum.index,
                             y=df_cumsum[plot_col_name],
                             mode='lines+markers',
                             marker=dict(size=3),
                             name=plot_col_name))
fig.show()

In [ ]:
df_returns_all_stocks_minute.sort_values(by='accepted_time', inplace=True)
plt.figure(figsize=(14,6));
# df_returns_all_stocks_minute.set_index('accepted_time')['accepted_time_t+1hr_%chg'].cumsum().plot(label='accepted_time+1hr')
# df_returns_all_stocks_minute.set_index('accepted_time')['accepted_time_t+3hr_%chg'].cumsum().plot(label='accepted_time+3hr')
# df_returns_all_stocks_minute.set_index('accepted_time')['accepted_time_t+5hr_%chg'].cumsum().plot(label='accepted_time+5hr')
# df_returns_all_stocks_minute.set_index('accepted_time')['accepted_time_t+7hr_%chg'].cumsum().plot(label='accepted_time+7hr')
# df_returns_all_stocks_minute.set_index('accepted_time')['accepted_time_t+10hr_%chg'].cumsum().plot(label='accepted_time+10hr')
# df_returns_all_stocks_minute.set_index('accepted_time')['accepted_time_t+12hr_%chg'].cumsum().plot(label='accepted_time+12hr')
df_returns_all_stocks_minute.set_index('date')['accepted_time_t+1_%chg'].cumsum().plot(label='accepted_time+1day')
df_returns_all_stocks_minute.set_index('date')['accepted_time_t+3_%chg'].cumsum().plot(label='accepted_time+3day')
df_returns_all_stocks_minute.set_index('date')['accepted_time_t+5_%chg'].cumsum().plot(label='accepted_time+5day')
df_returns_all_stocks_minute.set_index('date')['accepted_time_t+7_%chg'].cumsum().plot(label='accepted_time+7day')
df_returns_all_stocks_minute.set_index('date')['accepted_time_t+10_%chg'].cumsum().plot(label='accepted_time+10day')
df_returns_all_stocks_minute.set_index('date')['accepted_time_t+12_%chg'].cumsum().plot(label='accepted_time+12day')
plt.title('Cumulative Return on All 8-Ks with Minute Data, All SPAC Stocks');
plt.ylabel('Cumulative Return in Percentage Points (10 = 1000%)');
plt.xticks(rotation=45);
plt.legend();

In [ ]:
df_returns_all_warrants = df_returns_past_warrants_newticker.append(df_returns_current_warrants)
df_returns_all_warrants = df_returns_all_warrants.replace([np.inf, -np.inf], np.nan)
df_returns_all_warrants.sort_values(by='date', inplace=True)
df_returns_all_warrants.set_index('date')['open_close_t+1_%chg'].cumsum().plot(label='Open-Close T+1')
df_returns_all_warrants.set_index('date')['open_close_t+3_%chg'].cumsum().plot(label='Open-Close T+3')
df_returns_all_warrants.set_index('date')['open_close_t+5_%chg'].cumsum().plot(label='Open-Close T+5')
df_returns_all_warrants.set_index('date')['open_close_t+7_%chg'].cumsum().plot(label='Open-Close T+7')
df_returns_all_warrants.set_index('date')['open_close_t+10_%chg'].cumsum().plot(label='Open-Close T+10')
df_returns_all_warrants.set_index('date')['open_close_t+12_%chg'].cumsum().plot(label='Open-Close T+12')
df_returns_all_warrants.set_index('date')['open_completion_%chg'].cumsum().plot(label='Open-Completion')
plt.title('Cumulative Return on All 8-Ks, All SPAC Warrants');
plt.ylabel('Cumulative Return in Percentage Points (10 = 1000%)');
plt.xticks(rotation=45);
plt.legend();

In [ ]:
df_returns_all_warrants = df_returns_past_warrants_newticker.append(df_returns_current_warrants)
df_returns_all_warrants = df_returns_all_warrants.replace([np.inf, -np.inf], np.nan)
df_returns_all_warrants.sort_values(by='date', inplace=True)
plot_col_names = ['open_close_t+1_%chg', 'open_close_t+3_%chg', 'open_close_t+5_%chg', 'open_close_t+7_%chg',
                  'open_close_t+10_%chg', 'open_close_t+12_%chg', 'open_completion_%chg']
df_cumsum = df_returns_all_warrants.set_index('date')[plot_col_names].cumsum()
min_date = df_cumsum[~df_cumsum['open_close_t+1_%chg'].isna()].index.min()
df_cumsum = df_cumsum[df_cumsum.index>=min_date]
fig = go.Figure()
fig.update_layout(title='Cumulative Return Trading on All 8-Ks: SPAC Warrants',
                  xaxis_title='Date',
                  yaxis_title='Cumulative Return in Percentage Points (10 = 1000%)')
for plot_col_name in plot_col_names:
    fig.add_trace(go.Scatter(x=df_cumsum.index,
                             y=df_cumsum[plot_col_name],
                             mode='lines+markers',
                             marker=dict(size=3),
                             name=plot_col_name))
fig.show()

In [ ]:
df_returns_all_warrants_minute.sort_values(by='accepted_time', inplace=True)
plt.figure(figsize=(10,6));
# df_returns_all_warrants_minute.set_index('accepted_time')['accepted_time_t+1hr_%chg'].cumsum().plot(label='accepted_time+1hr')
# df_returns_all_warrants_minute.set_index('accepted_time')['accepted_time_t+3hr_%chg'].cumsum().plot(label='accepted_time+3hr')
# df_returns_all_warrants_minute.set_index('accepted_time')['accepted_time_t+5hr_%chg'].cumsum().plot(label='accepted_time+5hr')
# df_returns_all_warrants_minute.set_index('accepted_time')['accepted_time_t+7hr_%chg'].cumsum().plot(label='accepted_time+7hr')
# df_returns_all_warrants_minute.set_index('accepted_time')['accepted_time_t+10hr_%chg'].cumsum().plot(label='accepted_time+10hr')
# df_returns_all_warrants_minute.set_index('accepted_time')['accepted_time_t+12hr_%chg'].cumsum().plot(label='accepted_time+12hr')
df_returns_all_warrants_minute.set_index('date')['accepted_time_t+1_%chg'].cumsum().plot(label='accepted_time+1day')
df_returns_all_warrants_minute.set_index('date')['accepted_time_t+3_%chg'].cumsum().plot(label='accepted_time+3day')
df_returns_all_warrants_minute.set_index('date')['accepted_time_t+5_%chg'].cumsum().plot(label='accepted_time+5day')
df_returns_all_warrants_minute.set_index('date')['accepted_time_t+7_%chg'].cumsum().plot(label='accepted_time+7day')
df_returns_all_warrants_minute.set_index('date')['accepted_time_t+10_%chg'].cumsum().plot(label='accepted_time+10day')
df_returns_all_warrants_minute.set_index('date')['accepted_time_t+12_%chg'].cumsum().plot(label='accepted_time+12day')
plt.title('Cumulative Return on All 8-Ks with Minute Data, All SPAC Warrants');
plt.ylabel('Cumulative Return in Percentage Points (10 = 1000%)');
plt.xticks(rotation=45);
plt.legend();

# Predict warrants return using 8-K content

In [ ]:
def remove_header_footer(text):
    # remove/replace some unicode characters
    text = text.replace('\t','')
    text = text.replace('\x93','"')
    text = text.replace('\x94','"')
    text = text.replace('”', '"') # weird unicode/ascii conversion issue
    text = text.replace('“', '"') # weird unicode/ascii conversion issue
    
    # remove everything in header and footer
    ind_start = text.find('financial accounting standards provided pursuant to section 13(a) of the exchange act')
    ind_end = text.find('signature pursuant to the requirements of the securities exchange act of 1934')
    text = text[ind_start:ind_end]
    
    # remove forward looking statement section
    FLS_START = [
    'forward-looking statements this current report',
    'forward looking statements certain statements',
    'forward-looking statements this report',
    'forward-looking statements the company makes',
    'forward-looking statements certain of the matters',
    'forward-looking statements this communication',
    'forward-looking statements this document'
    ]
    FLS_END = [
    'undue reliance should not be placed upon the forward-looking statements',
    'whether as a result of new information, future events or otherwise, except as required by law',
    'conditions or circumstances on which any such statement is based, except as required by applicable law',
    'whether as a result of new information, future events, or otherwise'
    ]
    for forward_start in FLS_START:
        for forward_end in FLS_END:
            ind_start = text.find(forward_start)
            ind_end = text.find(forward_end)
            if ind_start!=-1 and ind_end!=-1:
                text = text[0:ind_start] + text[ind_end+len(forward_end):]
    
    # additional text to remove
    text_to_remove = [
        'financial accounting standards provided pursuant to section 13(a) of the exchange act'
    ]
    for rm in text_to_remove:
        text = text.replace(rm,'')
    
    return text

def get_item_subheaders(text, subheaders_only):
    subheaders = re.findall('item [0-9]+\.[0-9]+', text)
    subheaders = list(collections.OrderedDict.fromkeys(subheaders)) # handle cases where subheader mentioned in content
    subtexts = []
    for i in range(0,len(subheaders)):
        if i+1==len(subheaders):
            subtext = text[text.find(subheaders[i]):]
        else:
            subtext = text[text.find(subheaders[i]):text.find(subheaders[i+1])]
        if 'item 9.01 financial statements and exhibits' in subtext:
            continue
        else:
            subtexts.append(subtext)
    
    # drop these items
    drop_item_list = ['item 9.01']
    
    if subheaders_only:
        return [x for x in subheaders if x not in drop_item_list]
    return subtexts

def count_keywords(x, keywords):
    count = 0
    for keyword in keywords:
        count = count + x.count(keyword)
    return count

def text_processing(text, item_features, stemming=True):
    subtexts = get_item_subheaders(text, subheaders_only=False)
    tokens = []
    for subtext in subtexts:
        # remove item subheaders. todo: probably should remove text of subheader too
        for item in item_features:
            subtext = subtext.replace(item,'')

        # tokenize, only keeping alphanumeric
        subtokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(subtext)

        # remove stop words
        stop_words = set(nltk.corpus.stopwords.words('english'))
        subtokens = [w for w in subtokens if not w in stop_words]

        # remove numbers and chinese/other non-english characters for now
        # (todo: revisit later for vote count processing)
        subtokens = [w for w in subtokens if w.encode('utf-8').isalpha()]

        # stemming
        if stemming:
            stems = [nltk.stem.porter.PorterStemmer().stem(w) for w in subtokens]
            tokens.extend(stems)
        else:
            tokens.extend(subtokens)

    processed_text = ' '.join(tokens)
    return processed_text

def add_subheader_item_features(df_ret, item_features):
    # item definitions: https://www.sec.gov/fast-answers/answersform8khtm.html
    # item faq: https://media2.mofo.com/documents/faq-form-8-k.pdf
    for col in item_features:
        df_ret[col] = 0
    for i in range(0, len(df_ret)):
        text = df_ret.loc[i]['text']
        items = get_item_subheaders(text, subheaders_only=True)
        for item in items:
            df_ret.loc[i,item] = 1
    return df_ret

def add_bagofwords_features(df_ret, vectorizer_type, response_variable):
    # process text and create bigram, trigram features
    corpus = []
    for text in df_ret['text']:
        corpus.append(text_processing(text))
    print('vectorizer:', vectorizer_type, '\n')
    if vectorizer_type=='CountVectorizer':
        vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 3)) # Total Frequency
    elif vectorizer_type=='TfidfVectorizer':
        vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(2, 3)) # TF-IDF
    else:
        print('vectorizer_type d.n.e')
    X_corpus = vectorizer.fit_transform(corpus)
    df_bagofwords = pd.DataFrame(X_corpus.toarray())
    df_bagofwords.columns = vectorizer.get_feature_names()
    
    # concatenate
    df_ret = pd.concat([df_ret, df_bagofwords], axis=1)

    # example features
    print('letter intent features:', [x for x in df_bagofwords.columns if 'letter intent' in x])

    # histogram
    (n, bins, patches) = plt.hist(df_ret[response_variable], bins=10, label=response_variable)
    plt.title('Histogram of ' + response_variable)
    plt.show()
    print('bins', np.round(bins,2))
    print('counts', n)
    
    return df_ret

def convert_vote_count_to_int(x):
    if '—' in x or '-' in x or 'n/a' in x:
        return 0
    try:
        x = int(x.replace(',',''))
    except:
        return np.nan
    return x

# does not handle: https://www.sec.gov/Archives/edgar/data/1704760/000161577419006723/s117785_8k.htm (no broker non-votes)
def parse_vote_results(x):
    # find strings in text. use first if multiple matches
    VOTE_HEADER = [
        'for against abstain broker non-votes',
        'for against abstain broker non-vote',
        'for against abstention broker non-votes',
        'for against abstention broker non-vote',
        'for against abstentions broker non-votes',
        'for against abstentions broker non-vote'
    ]
    # find phrases preceding vote results in text
    vote_strings = [vote_string for vote_string in VOTE_HEADER if vote_string in x['text']]

    # parse votes for, votes against, votes abstain, votes broker non votes
    if len(vote_strings)==0:
        return pd.Series([np.nan, np.nan, np.nan, np.nan])
    else:
        vote_string = vote_strings[0] # use first if multiple matches
        vote_index = x['text'].find(vote_string)
        vote_data = x['text'][(vote_index + len(vote_string)):].lstrip().split(' ')
        votes_for = convert_vote_count_to_int(vote_data[0])
        votes_against = convert_vote_count_to_int(vote_data[1])
        votes_abstain = convert_vote_count_to_int(vote_data[2])
        votes_broker_non_votes = convert_vote_count_to_int(vote_data[3])
        if np.isnan(votes_for) or np.isnan(votes_against) or np.isnan(votes_abstain) or np.isnan(votes_broker_non_votes):
            print('something wrong with parse_vote_results for', x.symbol, 'on', x.date)
        return pd.Series([votes_for, votes_against, votes_abstain, votes_broker_non_votes])
    return pd.Series([np.nan, np.nan, np.nan, np.nan])

def parse_redemptions(x):
    REDEMPTION_HEADER = [
        'in connection with the extension',
        'in connection with the closing',
        'in advance of the special meeting',
        'in connection with the special meeting',
        'exercised their right'
    ]
    for redemption_phrase in REDEMPTION_HEADER:
        redemption_sentence = [sentence for sentence in x.text.split('.') if
                               redemption_phrase in sentence
                               and ('redeem' in sentence or 'redemp' in sentence)]
        if len(redemption_sentence)>0:
            break
    if len(redemption_sentence)==0:
        return np.nan
    redemption_sentence = redemption_sentence[0].lstrip().replace(',','')
    redemption_sentence = re.sub(r'[\$]{1}[\d,]+\.?\d{0,2}', '', redemption_sentence) # replace $_#_
    shares_regex_strong = re.findall('[0-9]+ shares', redemption_sentence)
    shares_regex_weak = re.findall('[0-9]+', redemption_sentence)
    # case: just one number in sentence. assume this number is redemption number
    if len(shares_regex_weak)==1:
        shares = int(shares_regex_weak[0])
    # case: more than one number in sentence. if '[0-9]+ shares' then assume this is redemption number, otherwise nan
    elif len(shares_regex_weak)>1:
        if len(shares_regex_strong)==1:
            shares = int(shares_regex_strong[0].replace('shares','').replace(' ',''))
        else:
            shares = np.nan
    # case: no numbers in sentence.
    else:
        if 'none' in redemption_sentence:
            shares = 0
        else:
            shares = np.nan
    return shares

def compute_self_engineered_feature_metrics(df_ret, col_name, response_variable):
    print('count', col_name, '> 0:', np.sum(df_ret[col_name]>0),
          '; count', response_variable, '> 0:', len(df_ret[df_ret[col_name]>0][df_ret[response_variable]>0]),
          '; count unique symbols:', len(df_ret[df_ret[col_name]>0]['symbol'].unique()))

def add_self_engineered_features(df_ret, response_variable=None):    
    # keywords lists
    keywords_list_loi = [
    'entry into a letter of intent',
    'entry into a non-binding letter of intent',
    'enter into a letter of intent',
    'enter into a non-binding letter of intent',
    'entered into a letter of intent',
    'entered into a non-binding letter of intent',
    'entering into a letter of intent',
    'entering into a non-binding letter of intent',
    'execution of a letter of intent',
    'execution of a non-binding letter of intent',
    'execute a letter of intent',
    'execute a non-binding letter of intent',
    'executed a letter of intent',
    'executed a non-binding letter of intent',
    'executing a letter of intent',
    'executing a non-binding letter of intent'
    ]
    keywords_list_business_combination_agreement = [
    '("business combination agreement")',
    '(the "business combination agreement")',
    '("business combination")',
    '(the "business combination")',
    'entry into a definitive agreement',
    'enter into a definitive agreement',
    'entered into a definitive agreement',
    'entering into a definitive agreement'
    ]
    keywords_list_merger_agreement = ['(the "merger agreement")']
    keywords_list_purchase_agreement = ['(the "purchase agreement")']
    keywords_list_extension = [
    '(the "extension")',
    '(the "extension amendment")',
    'extended the termination date',
    'extend the date by which the company must consummate',
    'extend the date by which the company has to complete',
    '(the "extension amendment proposal")',
    '(the "extended termination date")'
    ]
    keywords_list_meeting = ['("special meeting")', '(the "meeting")']
    keywords_list_record = ['(the "record date")']
    keywords_list_consummation = [
    'announcing the consummation',
    'consummated the previously announced business combination'
    ]
    keywords_list_ipo = [
    'consummated its initial public offering ("ipo")',
    'consummated its initial public offering (the "ipo")',
    'consummated an initial public offering ("ipo")',
    'consummated an initial public offering (the "ipo")',
    'consummated the initial public offering ("ipo")',
    'consummated the initial public offering (the "ipo")',
    'completed its initial public offering ("ipo")',
    'completed its initial public offering (the "ipo")',
    'in connection with its initial public offering ("ipo") was declared effective',
    'in connection with its initial public offering (the "ipo") was declared effective',
    'consummated the ipo',
    'in connection with the closing of the ipo',
    'closing of the initial public offering (the "ipo")',
    'consummation of the ipo'
    ]
    keywords_list_trust = [
    'trust account'
    ]
    
    # keywords counts
    df_ret['keywords_loi'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_loi))
    df_ret['keywords_business_combination_agreement'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_business_combination_agreement))
    df_ret['keywords_merger_agreement'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_merger_agreement))
    df_ret['keywords_purchase_agreement'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_purchase_agreement))
    df_ret['keywords_extension'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_extension))
    df_ret['keywords_meeting'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_meeting))
    df_ret['keywords_record'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_record))
    df_ret['keywords_consummation'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_consummation))
    df_ret['keywords_ipo'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_ipo))
    df_ret['keywords_trust'] = df_ret.text.apply(lambda x: count_keywords(x, keywords_list_trust))
    
    # compute metrics
    if response_variable is not None:
        keywords_name_list = ['keywords_loi','keywords_business_combination_agreement','keywords_merger_agreement',
                              'keywords_purchase_agreement','keywords_extension','keywords_meeting','keywords_record',
                              'keywords_consummation','keywords_ipo','keywords_trust','item 5.07','item 3.01','item 2.03']
        for keywords_name in keywords_name_list:
            compute_self_engineered_feature_metrics(df_ret, keywords_name, response_variable)

    # vote results
    df_ret['votes_for'] = np.nan
    df_ret['votes_against'] = np.nan
    df_ret['votes_abstain'] = np.nan
    df_ret['votes_broker_non_votes'] = np.nan
    df_ret[['votes_for', 'votes_against', 'votes_abstain', 'votes_broker_non_votes']] = df_ret.apply(lambda x: parse_vote_results(x), axis=1)
    df_ret['vote_total'] = df_ret['votes_for'] + df_ret['votes_against'] + df_ret['votes_abstain'] + df_ret['votes_broker_non_votes']
    df_ret['%votes_for'] = df_ret['votes_for'] / df_ret['vote_total']
    df_ret['%vote_against'] = df_ret['votes_against'] / df_ret['vote_total']
    df_ret['%votes_abstain'] = df_ret['votes_abstain'] / df_ret['vote_total']
    df_ret['%votes_broker_non_votes'] = df_ret['votes_broker_non_votes'] / df_ret['vote_total']
    
    # shares redeemed
    df_ret['redeemed_shares'] = df_ret.apply(lambda x: parse_redemptions(x), axis=1)
    df_ret['%redeemed'] = df_ret['redeemed_shares'] / df_ret['vote_total']
    
    return df_ret

# Useful text:
# "Item 7.01 Regulation FD Disclosure": LOI, Business Combination Agreement, Business Combination consummation
# "Item 5.07 Submission of Matters to a Vote of Security Holders": vote extension, vote result
# key words LOI: definitive agreement, letter of intent
# key words business combination agreement: "business combination agreement", "business combination", "merger agreement", "purchase agreement"
# key words vote: ("special meeting"), (the "meeting"), (the "extension"), (the "extension amendment"), (the "record date")
# key words other: (the "closing date")

# Not useful:
# dropped "item 9.01 financial statements and exhibits"
# maybe drop "Forward-Looking Statements", "No Offer or Solicitation"
# maybe drop "Item 5.02. Departure of Directors or Certain Officers; Election of Directors; Appointment of Certain Officers; Compensatory Arrangements of Certain Officers."

# other notes:
# 8-Ks aren't actually iid, maybe symbols with previous positive reactions are more likely to positive
#    - example of bad reaction KXIN Form 25 https://www.sec.gov/Archives/edgar/data/1713539/000135445719000234/0001354457-19-000234-index.htm
#    - another example: NSCO Form 25 https://www.sec.gov/Archives/edgar/data/1709682/000087666119000802/0000876661-19-000802-index.htm
# stratify by sector
# add class weight instead of cv with f1 score
# maybe create set of features using all legal terms in (), e.g. (the "special meeting")

In [ ]:
# data processing inputs
y_variable = 'open_close_t+5_%chg' # t+1, t+2,..., or t+12

# get all warrants and drop nan and corrupt symbols
# todo: merge newticker and oldticker df_returns_past_warrants properly. use newticker for now
df_returns_warrants = df_returns_past_warrants_newticker.append(df_returns_current_warrants)
df_returns_warrants.drop(columns=['letter_of_intent_found',
                                  'business_combination_agreement_found',
                                  'form','open_completion_%chg'], inplace=True)

print('count 8-Ks:', len(df_returns_warrants))
df_returns_warrants.dropna(inplace=True)
print('count 8-Ks after dropping nan prices:', len(df_returns_warrants))
# remove corrupt symbols
df_returns_warrants = df_returns_warrants[~df_returns_warrants.symbol.isin(['ACELW','LAZYW'])]
print('count 8-Ks after dropping corrupt symbols:', len(df_returns_warrants), '\n')
df_returns_warrants.sort_values(by='accepted_time', inplace=True) # need to sort for TimeSeriesSplit
df_returns_warrants.reset_index(inplace=True, drop=True)

print('predicting:', y_variable)

# define item features
item_features = ['item 1.01','item 1.02','item 1.03','item 1.04','item 2.01','item 2.02','item 2.03','item 2.04',
                 'item 2.05','item 2.06','item 3.01','item 3.02','item 3.03','item 4.01','item 4.02','item 5.01',
                 'item 5.02','item 5.03','item 5.04','item 5.05','item 5.06','item 5.07','item 5.08','item 6.01',
                 'item 6.02','item 6.03','item 6.04','item 6.05','item 7.01','item 8.01']

# remove header and footer
df_returns_warrants['text'] = df_returns_warrants.text.apply(lambda x: remove_header_footer(x))

# add subheader item binary features
df_returns_warrants = add_subheader_item_features(df_ret=df_returns_warrants, item_features=item_features)

# add bag of words features
# df_returns_warrants = add_bagofwords_features(df_ret=df_returns_warrants,
#                                               vectorizer_type='CountVectorizer', # CountVectorizer or TfidfVectorizer
#                                               response_variable=y_variable)

# add self engineered features
df_returns_warrants = add_self_engineered_features(df_ret=df_returns_warrants, response_variable=y_variable)

In [ ]:
# study different keywords

def string_match_test(test_strings):
    print('strings match:', test_strings, '\n')
    df_matched = df_returns_warrants[(df_returns_warrants.text.str.contains('|'.join(test_strings))) &
                                     (df_returns_warrants['item 2.03']==0)]
    print('count 8-Ks matched:', len(df_matched))
    df_matched.set_index('date')['open_close_t+1_%chg'].cumsum().plot(label='Open-Close T+1')
    df_matched.set_index('date')['open_close_t+3_%chg'].cumsum().plot(label='Open-Close T+3')
    df_matched.set_index('date')['open_close_t+5_%chg'].cumsum().plot(label='Open-Close T+5')
    df_matched.set_index('date')['open_close_t+7_%chg'].cumsum().plot(label='Open-Close T+7')
    df_matched.set_index('date')['open_close_t+10_%chg'].cumsum().plot(label='Open-Close T+10')
    df_matched.set_index('date')['open_close_t+12_%chg'].cumsum().plot(label='Open-Close T+12')
    plt.ylabel('Cumulative Return in Percentage Points (10 = 1000%)');
    plt.xticks(rotation=45);
    plt.legend();
    plt.show();

# loi
string_match_test(test_strings=[
    'entry into a letter of intent',
    'entry into a non-binding letter of intent',
    'entered into a letter of intent',
    'entered into a non-binding letter of intent',
    'entering into a letter of intent',
    'entering into a non-binding letter of intent',
    'execute a letter of intent',
    'execute a non-binding letter of intent',
    'executed a letter of intent',
    'executed a non-binding letter of intent',
    'executing a letter of intent',
    'executing a non-binding letter of intent',
    'execution of a letter of intent',
    'execution of a non-binding letter of intent',
])
    
# bca
string_match_test(test_strings=[
    '("business combination agreement")',
    '(the "business combination agreement")',
    '("business combination")',
    '(the "business combination")',
    'entry into a definitive agreement',
    'enter into a definitive agreement',
    'entered into a definitive agreement',
    'entering into a definitive agreement',
    'business combination proposal was approved',
#     'entered into a stock and unit purchase agreement',
#     'entered into a securities purchase agreement',
#     'entered into an agreement and plan of merger',
#     'entered into a share exchange agreement',
])

# extension
keywords_list_extension = [
'(the "extension")',
'(the "extension amendment")',
'extended the termination date',
'extend the date by which the company must consummate',
'extend the date by which the company has to complete',
'(the "extension amendment proposal")',
'(the "extended termination date")'
]
string_match_test(test_strings=keywords_list_extension)

# consummation
string_match_test(test_strings=[
    'announcing the consummation',
    'consummated the previously announced business combination',
    'consummated the remainder of the transactions contemplated by the business combination',
    'consummation of the transactions',
])
                  
# investor presentation
string_match_test(test_strings=[
    'form of presentation',
    'furnished an investor presentation',
    'updated investor presentation',
    'investor presentation are furnished',
    'form of investor presentation',
    'update the investor presentation',
])
string_match_test(test_strings=[ # this could be a good one
    'investor presentation'
])

# trust account
string_match_test(test_strings=[
    '(the "trust account")',
    '(the "contribution")',
    '(the "revised contribution")',
])
string_match_test(test_strings=[ # this could be a good one
    'trust account',
])

# form 25-nse
string_match_test(test_strings=[ # expect this to be a sell signal
    'form 25',
])

# consummate IPO
keywords_list_ipo = [
'consummated its initial public offering ("ipo")',
'consummated its initial public offering (the "ipo")',
'consummated an initial public offering ("ipo")',
'consummated an initial public offering (the "ipo")',
'consummated the initial public offering ("ipo")',
'consummated the initial public offering (the "ipo")',
'completed its initial public offering ("ipo")',
'completed its initial public offering (the "ipo")',
'in connection with its initial public offering ("ipo") was declared effective',
'in connection with its initial public offering (the "ipo") was declared effective',
'consummated the ipo',
'in connection with the closing of the ipo',
'closing of the initial public offering (the "ipo")',
'consummation of the ipo'
]
string_match_test(test_strings=keywords_list_ipo)

# pipe investor
string_match_test(test_strings=[
    'pipe',
])

In [ ]:
# randomly sample 8-Ks for populating test cases
# path_saved_8K = 'data/sec_filings_df'
# files_8Ks = [f for f in os.listdir(path_saved_8K) if os.path.isfile(os.path.join(path_saved_8K, f))]
# df_8Ks = pd.DataFrame()
# for file in files_8Ks:
#     df_8K = pd.read_csv(path_saved_8K+'/'+file)
#     df_8K['symbol'] = file.split('_')[0]
#     df_8Ks = df_8Ks.append(df_8K)
# df_8Ks.drop_duplicates(subset=['date','accepted_time','text'], inplace=True)
# df_8Ks.reset_index(drop=True, inplace=True)
# R = np.random.RandomState(123)
# random_inds = R.random_integers(0, len(df_8Ks), 110)
# df_8Ks[df_8Ks.index.isin(random_inds[100:110])]

# check single test case
# spac_list_temp = process_current_spacs(file_path_current='data/spac_list_temp.csv', write=False)
# temp = get_forms_text(company_name=spac_list_temp.title[0], cik_id=spac_list_temp.cik[0], form_type='8-K')
# temp['symbol'] = spac_list_temp.ticker[0]
# temp['text'] = temp.text.apply(lambda x: remove_header_footer(x))
# temp = add_subheader_item_features(df_ret=temp, item_features=item_features)
# temp = add_self_engineered_features(df_ret=temp)
# display(temp.head(1))

#### Binary Classification

In [ ]:
def apply_lsa_dim_reduction(X, n_lsa):
    lsa = TruncatedSVD(n_components=n_lsa, n_iter=10, random_state=123)
    X = pd.DataFrame(lsa.fit_transform(X), columns=['lsa'+str(i) for i in range(0,n_lsa)])
    print('count feature after LSA:', len(X.columns), '\n')
    return X

def binary_classification_eval_metrics(y_actual, y_pred):
    print('% predicted positive labels:', np.round(sum(y_pred)/len(y_pred), 2))
    print('% actual positive labels:', np.round(sum(y_actual)/len(y_actual), 2))
    cm = confusion_matrix(y_actual, y_pred)
    print('confusion matrix:\n', cm)
    print(classification_report(y_actual, y_pred, target_names=['0','1']))

def binary_classification_report(model, X_train, X_test, y_train, y_test, feature_importance=True):
    print('############\nTraining set\n############')
    y_train_pred = model.predict(X_train)
    binary_classification_eval_metrics(y_train, y_train_pred)
    metrics.plot_roc_curve(model, X_train, y_train);
    plt.title('ROC curve (train)')
    plt.show()
    metrics.plot_precision_recall_curve(model, X_train, y_train)
    plt.title('Precision-recall curve (train)');
    plt.show()
    
    print('#########\nTest set\n#########')
    y_test_pred = model.predict(X_test)
    binary_classification_eval_metrics(y_test, y_test_pred)
    metrics.plot_roc_curve(model, X_test, y_test);
    plt.title('ROC curve (test)')
    plt.show()
    metrics.plot_precision_recall_curve(model, X_test, y_test)
    plt.title('Precision-recall curve (test)');
    plt.show()
    
    print('#########\nAll data\n#########')
    y_all_pred = model.predict(X)
    binary_classification_eval_metrics(y, y_all_pred)
    metrics.plot_roc_curve(model, X, y);
    plt.title('ROC curve (all)')
    plt.show()
    metrics.plot_precision_recall_curve(model, X, y)
    plt.title('Precision-recall curve (all)');
    plt.show()
    
    # feature importance
    if feature_importance:
        try:
            coefs = model.coef_[0]
        except:
            coefs = model.feature_importances_
        indices = np.argsort(np.abs(coefs))[::-1]
        count_nonzero_features = len(X.columns[coefs!=0])
        print('count nonzero features:', count_nonzero_features)
        if count_nonzero_features < 10:
            top_n_features = count_nonzero_features
        else:
            top_n_features = 10
        plt.bar(range(top_n_features), coefs[indices[:top_n_features]], align='center')
        plt.xticks(range(top_n_features), X.columns[indices[:top_n_features]], rotation=45, ha='right')
        plt.subplots_adjust(bottom=0.3)
        plt.title('Feature importance')
        plt.show()

In [ ]:
# inputs
label_threshold = 0
min_word_freq = 0
n_lsa = None # None or int

# prep data
cols_drop = ['symbol','date','accepted_time','text','url','votes_for','votes_against',
             'votes_abstain','votes_broker_non_votes','vote_total','%votes_for',
             '%votes_abstain','%votes_broker_non_votes','redeemed_shares']
cols_drop.extend([col for col in df_returns_warrants.columns if '%chg' in col])
X = df_returns_warrants.drop(cols_drop, axis=1)
X[['%vote_against','%redeemed']] = X[['%vote_against','%redeemed']].fillna(0)
print('count features:', len(X.columns))
X = X[X.columns[X.sum() >= min_word_freq].tolist()] # min word frequency
print('count features > min_word_freq:', len(X.columns))
if n_lsa is not None:
    X = apply_lsa_dim_reduction(X, n_lsa) # lsa dimension reduction
y = np.where(df_returns_warrants[y_variable] > label_threshold, 1, 0) # label threshold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print('training dim:', X_train.shape)
print('testing dim:', X_test.shape, '\n')
print('count 0:', np.sum(y==0))
print('count 1:', np.sum(y==1))

In [ ]:
print('Naive rule based model\n')

def naive_rule(x):
    if ~np.isnan(x['%vote_against']) and (x['%vote_against'] > .10):
        return 0
    elif x['keywords_ipo'] > 0:
        return 0
    else:
        if (x['keywords_loi'] > 0) and (x['item 2.03'] == 0):
            return 1
        elif (x['keywords_business_combination_agreement'] > 0) and (x['item 2.03'] == 0):
            return 1
        elif (x['keywords_consummation'] > 0) and (x['item 2.03'] == 0):
            return 1
        elif (x['keywords_extension'] > 0) and (x['item 2.03'] == 0):
            return 1
        elif (x['keywords_trust'] > 0) and (x['item 2.03'] == 0):
            return 1
        else:
            return 0

y_train_pred = X_train.apply(lambda x: naive_rule(x), axis=1)
y_test_pred = X_test.apply(lambda x: naive_rule(x), axis=1)
y_all_pred = X.apply(lambda x: naive_rule(x), axis=1)

print('############\nTraining set\n############')
binary_classification_eval_metrics(y_train, y_train_pred)

print('#########\nTest set\n#########')
binary_classification_eval_metrics(y_test, y_test_pred)

print('#########\nAll data\n#########')
binary_classification_eval_metrics(y, y_all_pred)
print('Count examples:', len(y))
print('Count positive labels:', y_all_pred.sum())
count_months = int(np.round((dt.strptime(df_returns_warrants.date.max(),'%Y-%m-%d') -
                             dt.strptime(df_returns_warrants.date.min(),'%Y-%m-%d')).days / 365. * 12, 0))
print('Count months:', count_months)
print('Trades / month:', np.round(y_all_pred.sum()/count_months, 1))

df_pred_pos = df_returns_warrants.loc[np.where(y_all_pred==1)[0],]
print(y_variable, 'sum:', np.round(df_pred_pos[y_variable].sum(), 2))
# todo: calculate sharpe and sortino ratios properly
print(y_variable, 'mean:', np.round(df_pred_pos[y_variable].mean(), 2))
print(y_variable, 'sd:', np.round(df_pred_pos[y_variable].std(), 2))
print(y_variable, 'downside sd:', np.round(df_pred_pos[y_variable][df_pred_pos[y_variable]<0].std(), 2))

df_pred_pos.set_index('date')['open_close_t+1_%chg'].cumsum().plot(label='Open-Close T+1')
df_pred_pos.set_index('date')['open_close_t+3_%chg'].cumsum().plot(label='Open-Close T+3')
df_pred_pos.set_index('date')['open_close_t+5_%chg'].cumsum().plot(label='Open-Close T+5')
df_pred_pos.set_index('date')['open_close_t+7_%chg'].cumsum().plot(label='Open-Close T+7')
df_pred_pos.set_index('date')['open_close_t+10_%chg'].cumsum().plot(label='Open-Close T+10')
df_pred_pos.set_index('date')['open_close_t+12_%chg'].cumsum().plot(label='Open-Close T+12')
plt.title('Cumulative Return on Predicted Positive Labels, All Warrants');
plt.ylabel('Cumulative Return in Percentage Points (10 = 1000%)');
plt.xticks(rotation=45);
plt.legend();
plt.show();

cols_plot = ['keywords_loi',
#              'keywords_business_combination_agreement',
#              'keywords_consummation',
             'keywords_extension',
             'keywords_trust']
for col in cols_plot:
    df_pred_pos[df_pred_pos[col]>0].set_index('date')['open_close_t+1_%chg'].cumsum().plot(label='Open-Close T+1')
    df_pred_pos[df_pred_pos[col]>0].set_index('date')['open_close_t+3_%chg'].cumsum().plot(label='Open-Close T+3')
    df_pred_pos[df_pred_pos[col]>0].set_index('date')['open_close_t+5_%chg'].cumsum().plot(label='Open-Close T+5')
    df_pred_pos[df_pred_pos[col]>0].set_index('date')['open_close_t+7_%chg'].cumsum().plot(label='Open-Close T+7')
    df_pred_pos[df_pred_pos[col]>0].set_index('date')['open_close_t+10_%chg'].cumsum().plot(label='Open-Close T+10')
    df_pred_pos[df_pred_pos[col]>0].set_index('date')['open_close_t+12_%chg'].cumsum().plot(label='Open-Close T+12')
    plt.title('Cumulative Return on Predicted Positive Labels, All Warrants, '+col);
    plt.ylabel('Cumulative Return in Percentage Points (10 = 1000%)');
    plt.xticks(rotation=45);
    plt.legend();
    plt.show();
    
# gini-like metric for predicted positive labels
keywords_name_list = ['keywords_loi','keywords_extension','keywords_trust']
for keywords_name in keywords_name_list:
    compute_self_engineered_feature_metrics(df_pred_pos, keywords_name, y_variable)

# todo: weird ones to look at
# GTYH: df_returns_warrants[df_returns_warrants.symbol=='GTYHW'].loc[115,'text']...somehow doesn't match up with https://www.sec.gov/Archives/edgar/data/1682325/000114420419014675/tv516340_8k12ba.htm

In [ ]:
# print('loi symbols:', df_returns_warrants[(df_returns_warrants.keywords_loi>0)].symbol.unique())
# print('\nbusiness combination agreement symbols:', df_returns_warrants[(df_returns_warrants.keywords_business_combination_agreement>0)].symbol.unique())
# print('\nextension symbols:', df_returns_warrants[(df_returns_warrants.keywords_extension>0)].symbol.unique())
# print('\nconsummation symbols:', df_returns_warrants[(df_returns_warrants.keywords_consummation>0)].symbol.unique())
# print('\ntrust symbols:', df_returns_warrants[(df_returns_warrants.keywords_trust>0)].symbol.unique())

In [ ]:
# df_pred_pos.groupby('symbol').agg({'accepted_time':'count'}).sort_values(by='accepted_time', ascending=False).head()

In [ ]:
# df_pred_pos.sort_values(by='open_close_t+3_%chg').head()

In [ ]:
print('Logistic Regression\n')

# CV
model_lr = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
C = [.01, .1, 1, 10, 100, 1000]
# C = np.linspace(.5, 1.5, 10)
param_grid = dict(C=C)
kfold = TimeSeriesSplit(n_splits=5) # in each split test indices must be higher than before
grid_search = GridSearchCV(model_lr, param_grid, scoring='precision', cv=kfold, verbose=0, n_jobs=5)
grid_result = grid_search.fit(X_train, y_train)
print('Best 5-fold CV (TimeSeriesSplit) score: %f using %s' %
      (grid_result.best_score_, grid_result.best_params_), '\n')
reg_path = pd.DataFrame({'score': grid_result.cv_results_['mean_test_score'], 'C': C})
reg_path['C'] = np.log10(reg_path['C'])
reg_path.plot(x='C')
plt.title('CV path')
plt.xlabel('log10(C)')
plt.show()

# model
model_lr = LogisticRegression(penalty='l1', solver='liblinear', C=grid_result.best_params_['C'], max_iter=1000)
model_lr.fit(X_train, y_train)
binary_classification_report(model_lr, X_train, X_test, y_train, y_test)

In [ ]:
print('Decision Tree\n')

# CV
model_dt = DecisionTreeClassifier()
max_depth = list(range(1, 20))
# min_samples_split = [.001, .01, .05, .1]
# min_samples_leaf = [.001, .01, .05, .1]
param_grid = dict(max_depth=max_depth) #, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
kfold = TimeSeriesSplit(n_splits=5) # in each split test indices must be higher than before
grid_search = GridSearchCV(model_dt, param_grid, scoring='f1', cv=kfold, verbose=0, n_jobs=5)
grid_result = grid_search.fit(X_train, y_train)
print('Best 5-fold CV (TimeSeriesSplit) score: %f using %s' %
      (grid_result.best_score_, grid_result.best_params_), '\n')
reg_path = pd.DataFrame({'score': grid_result.cv_results_['mean_test_score'], 'max_depth': max_depth})
reg_path.plot(x='max_depth')
plt.title('CV path')
plt.xlabel('max_depth')
plt.show()

# model
model_dt = DecisionTreeClassifier(max_depth=grid_result.best_params_['max_depth'])
#                                   min_samples_split=grid_result.best_params_['min_samples_split'],
#                                   min_samples_leaf=grid_result.best_params_['min_samples_leaf'])
model_dt.fit(X_train, y_train)
binary_classification_report(model_dt, X_train, X_test, y_train, y_test)

# tree diagram
tree_data = export_graphviz(model_dt, out_file=None, feature_names=X.columns, class_names=['-1','1'],
                            filled=True, rounded=True, rotate=True, impurity=False)
graph = graphviz.Source(tree_data)
graph

In [ ]:
print('Random Forest\n')

# CV
model_rf = RandomForestClassifier()
n_estimators = list(range(1, 5))
max_depth = list(range(1, 5))
min_samples_leaf = [.01, .05, .1, .2]
max_features = [.01, .05, .1]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth,
                  min_samples_leaf=min_samples_leaf, max_features=max_features)
kfold = TimeSeriesSplit(n_splits=5) # in each split test indices must be higher than before
grid_search = GridSearchCV(model_rf, param_grid, scoring='f1', cv=kfold, verbose=0, n_jobs=5)
grid_result = grid_search.fit(X_train, y_train)
print('Best 5-fold CV (TimeSeriesSplit) score: %f using %s' %
      (grid_result.best_score_, grid_result.best_params_), '\n')

# model
model_rf = RandomForestClassifier(n_estimators=grid_result.best_params_['n_estimators'],
                                  max_depth=grid_result.best_params_['max_depth'],
                                  min_samples_leaf=grid_result.best_params_['min_samples_leaf'],
                                  max_features=grid_result.best_params_['max_features'])
model_rf.fit(X_train, y_train)
binary_classification_report(model_rf, X_train, X_test, y_train, y_test)

In [ ]:
print('SVM\n')

kernel_type = 'linear' # linear, rbf, poly

# CV
model_svm = SVC(kernel=kernel_type)
C = [.01, .1, 1, 10, 100, 1000, 10000]
# C = [.6, .7, .8, .9, 1, 1.1, 1.2]
param_grid = dict(C=C)
kfold = TimeSeriesSplit(n_splits=5) # in each split test indices must be higher than before
grid_search = GridSearchCV(model_svm, param_grid, scoring='f1', cv=kfold, verbose=0, n_jobs=5)
grid_result = grid_search.fit(X_train, y_train)
print('Best 5-fold CV (TimeSeriesSplit) score: %f using %s' %
      (grid_result.best_score_, grid_result.best_params_), '\n')
reg_path = pd.DataFrame({'score': grid_result.cv_results_['mean_test_score'], 'C': C})
reg_path['C'] = np.log10(reg_path['C'])
reg_path.plot(x='C')
plt.title('CV path')
plt.xlabel('log10(C)')
plt.show()

# model
model_svm = SVC(kernel=kernel_type, C=grid_result.best_params_['C'])
model_svm.fit(X_train, y_train)
binary_classification_report(model_svm, X_train, X_test, y_train, y_test, feature_importance=False)

#### 3-Class Classification

In [ ]:
def get_multi_labels(x):
    if x < first_label_threshold:
        return -1
    elif x > second_label_threshold:
        return 1
    else:
        return 0
    
def plot_roc_multi(model_multi, X_input, y_input):
    y_binarized = preprocessing.label_binarize(y_input, classes=[-1, 0, 1])
    n_classes = y_binarized.shape[1]
    fpr = dict()
    tpr = dict()
    from itertools import cycle
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = metrics.roc_curve(y_binarized[:, i], model_multi.predict_proba(X_input)[:, i])
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])
    colors = cycle(['blue', 'red', 'green'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=1,
                 label='ROC curve of class {0} (AUC = {1:0.2f})'
                 ''.format(i-1, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()
    
def multi_classification_eval_metrics(y_actual, y_pred):
    print('% predicted (actual) -1 labels: {} ({})'.format(np.round(sum(y_pred==-1)/len(y_pred), 2),
                                                           np.round(sum(y_actual==-1)/len(y_actual), 2)))
    print('% predicted (actual) 0 labels: {} ({})'.format(np.round(sum(y_pred==0)/len(y_pred), 2),
                                                          np.round(sum(y_actual==0)/len(y_actual), 2)))
    print('% predicted (actual) 1 labels: {} ({})'.format(np.round(sum(y_pred==1)/len(y_pred), 2),
                                                          np.round(sum(y_actual==1)/len(y_actual), 2)))
    cm = confusion_matrix(y_actual, y_pred)
    print('confusion matrix:\n', cm)
    print(classification_report(y_actual, y_pred, target_names=['-1','0','1']))
    
def multi_classification_report(model_multi, X_train, X_test, y_train, y_test, feature_importance=True):
    print('############\nTraining set\n############')
    y_pred_train = model_multi.predict(X_train)
    multi_classification_eval_metrics(y_train, y_pred_train)
    plot_roc_multi(model_multi, X_train, y_train)

    print('#########\nTest set\n#########')
    y_pred = model_multi.predict(X_test)
    multi_classification_eval_metrics(y_test, y_pred)
    plot_roc_multi(model_multi, X_test, y_test)

    # feature importance
    if feature_importance:
        try:
            coefs = model_multi.coef_[0]
        except:
            coefs = model_multi.feature_importances_
        indices = np.argsort(np.abs(coefs))[::-1]
        print('count nonzero features:', len(X.columns[coefs!=0]))
        top_n_features = 20 #len(X.columns[coefs!=0])
        plt.bar(range(top_n_features), coefs[indices[:top_n_features]], align='center')
        plt.xticks(range(top_n_features), X.columns[indices[:top_n_features]], rotation=45, ha='right')
        plt.subplots_adjust(bottom=0.3)
        plt.title('Feature importance')
        plt.show()

In [ ]:
# inputs
first_label_threshold = -0.05 #-0.17
second_label_threshold = 0.05 #0.38
min_word_freq = 0
n_lsa = None

# prep data
X = df_returns_warrants.drop([y_variable,'symbol','date','text'], axis=1)
print('count features:', len(X.columns))
X = X[X.columns[X.sum() >= min_word_freq].tolist()] # min word frequency
print('count features > min_word_freq:', len(X.columns))
if n_lsa is not None:
    X = apply_lsa_dim_reduction(X, n_lsa) # lsa dimension reduction
y = df_returns_warrants[y_variable].map(get_multi_labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print('training dim:', X_train.shape)
print('testing dim:', X_test.shape, '\n')
print('count -1:', np.sum(y==-1))
print('count 0:', np.sum(y==0))
print('count 1:', np.sum(y==1), '\n')

In [ ]:
print('Logistic Regression\n')

# CV
model_lr_multi = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial')
C = [.01, .1, 1, 10, 50, 100, 1000]
param_grid = dict(C=C)
kfold = TimeSeriesSplit(n_splits=5) # in each split test indices must be higher than before
grid_search = GridSearchCV(model_lr_multi, param_grid, scoring=metrics.make_scorer(metrics.f1_score, average='macro'),
                           cv=kfold, verbose=0, n_jobs=5)
grid_result = grid_search.fit(X_train, y_train)
print('Best 5-fold CV (TimeSeriesSplit) score: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
reg_path = pd.DataFrame({'score': grid_result.cv_results_['mean_test_score'], 'C': C})
reg_path['C'] = np.log10(reg_path['C'])
reg_path.plot(x='C')
plt.title('CV path')
plt.xlabel('log10(C)')
plt.show()

# model
model_lr_multi = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial',
                                    C=grid_result.best_params_['C'])
model_lr_multi.fit(X_train, y_train)
multi_classification_report(model_lr_multi, X_train, X_test, y_train, y_test)

In [ ]:
print('Decision Tree\n')

# CV
model_dt_multi = DecisionTreeClassifier()
max_depth = list(range(1, 10))
param_grid = dict(max_depth=max_depth)
kfold = TimeSeriesSplit(n_splits=5) # in each split test indices must be higher than before
grid_search = GridSearchCV(model_dt_multi, param_grid, scoring='f1_macro', cv=kfold, verbose=0, n_jobs=5)
y_train_binarized = preprocessing.label_binarize(y_train, classes=[-1, 0, 1])
grid_result = grid_search.fit(X_train, y_train_binarized)
print('Best 5-fold CV (TimeSeriesSplit) score: %f using %s' %
      (grid_result.best_score_, grid_result.best_params_), '\n')
reg_path = pd.DataFrame({'score': grid_result.cv_results_['mean_test_score'], 'max_depth': max_depth})
reg_path.plot(x='max_depth')
plt.title('CV path')
plt.xlabel('max_depth')
plt.show()

# model
model_dt_multi = DecisionTreeClassifier(max_depth=grid_result.best_params_['max_depth'])
model_dt_multi.fit(X_train, y_train)
multi_classification_report(model_dt_multi, X_train, X_test, y_train, y_test)

# tree diagram
# tree_data = export_graphviz(model_dt_multi, out_file=None, feature_names=X.columns, class_names=['-1','0','1'],
#                             filled=True, rounded=True, rotate=True, impurity=False)
# graph = graphviz.Source(tree_data)
# graph

In [ ]:
print('Random Forest\n')

# CV
model_rf_multi = RandomForestClassifier()
n_estimators = list(range(1, 10))
max_depth = list(range(1, 5))
min_samples_leaf = [.01, .05, .1, .2, .3]
max_features = [.05, .25, .5, .75]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth,
                  min_samples_leaf=min_samples_leaf, max_features=max_features)
kfold = TimeSeriesSplit(n_splits=5) # in each split test indices must be higher than before
grid_search = GridSearchCV(model_rf_multi, param_grid, scoring='f1_macro', cv=kfold, verbose=0, n_jobs=5)
y_train_binarized = preprocessing.label_binarize(y_train, classes=[-1, 0, 1])
grid_result = grid_search.fit(X_train, y_train_binarized)
print('Best 5-fold CV (TimeSeriesSplit) score: %f using %s' %
      (grid_result.best_score_, grid_result.best_params_), '\n')

# model
model_rf_multi = RandomForestClassifier(n_estimators=grid_result.best_params_['n_estimators'],
                                        max_depth=grid_result.best_params_['max_depth'],
                                        min_samples_leaf=grid_result.best_params_['min_samples_leaf'],
                                        max_features=grid_result.best_params_['max_features'])
model_rf_multi.fit(X_train, y_train)
multi_classification_report(model_rf_multi, X_train, X_test, y_train, y_test)

In [ ]:
print('SVM\n')

kernel_type = 'linear' # linear, rbf, poly

# CV
model_svm_multi = SVC(kernel=kernel_type, probability=True)
C = [.01, .1, 1, 10, 50, 100, 150, 1000, 10000]
param_grid = dict(C=C)
kfold = TimeSeriesSplit(n_splits=5) # in each split test indices must be higher than before
grid_search = GridSearchCV(model_svm_multi, param_grid, scoring='f1_macro', cv=kfold, verbose=0, n_jobs=5)
grid_result = grid_search.fit(X_train, y_train)
print('Best 5-fold CV (TimeSeriesSplit) score: %f using %s' %
      (grid_result.best_score_, grid_result.best_params_), '\n')
reg_path = pd.DataFrame({'score': grid_result.cv_results_['mean_test_score'], 'C': C})
reg_path['C'] = np.log10(reg_path['C'])
reg_path.plot(x='C')
plt.title('CV path')
plt.xlabel('log10(C)')
plt.show()

# model
model_svm_multi = SVC(kernel=kernel_type, C=grid_result.best_params_['C'], probability=True)
model_svm_multi.fit(X_train, y_train)
multi_classification_report(model_svm_multi, X_train, X_test, y_train, y_test, feature_importance=False)